In [1]:
# SETUP PATHS

WORKSPACE_PATH = 'workspace'
SCRIPTS_PATH = 'scripts'
APIMODEL_PATH = 'models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = PRETRAINED_MODEL_PATH+'/pipeline_config.config'

In [3]:
# Split data Train & Test 80:20

import pandas as pd
from sklearn.model_selection import train_test_split

data_all= pd.read_csv(ANNOTATION_PATH + '/annotation.csv')

X=data_all.drop('class', axis=1)
y=data_all['class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=1010)

y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

data_train=pd.merge(X_train,y_train,left_index=True,right_index=True)
data_test=pd.merge(X_test,y_test,left_index=True,right_index=True)

data_train.to_csv(ANNOTATION_PATH + '/train.csv')
data_test.to_csv(ANNOTATION_PATH + '/test.csv')

In [4]:
# LABEL MAP

labels = [
    {'id':1, 'name':'car'},
    {'id':2, 'name':'motorcycle'},
    {'id':3, 'name':'person'},
    {'id':4, 'name':'pothole'},
]

with open(ANNOTATION_PATH + '\anotasi\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [5]:
# CREATE tfrecord

def class_text_to_int(row_label):
    if row_label =='car':
        return 1
    elif row_label =='motorcycle':
        return 2
    elif row_label =='pothole':
        return 3
    elif row_label =='person':
        return 4
    else:
        return None

In [6]:
get_ipython().system("python {SCRIPTS_PATH + '/generate_tfrecord.py'} --csv_input {ANNOTATION_PATH + '/anotasi/train.csv'} --image_dir {IMAGE_PATH + '/aug_images'} --output_path {ANNOTATION_PATH + '/anotasi/train.record'}")
get_ipython().system("python {SCRIPTS_PATH + '/generate_tfrecord.py'} --csv_input {ANNOTATION_PATH + '/anotasi/test.csv'} --image_dir {IMAGE_PATH + '/aug_images'} --output_path {ANNOTATION_PATH + '/anotasi/test.record'}")

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [9]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 300
       width: 300
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9997000098228455
         center: true
         scale: true
         epsilon: 0.0010000000474974513
         train: true
       }
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
       unmatched_threshold: 0.5
       ignore_thresholds: false
       negativ

In [10]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [11]:
pipeline_config.model.ssd.num_classes = 4
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH +'/pre-trained-models/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/anotasi/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/anotasi/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/anotasi/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/anotasi/test.record']

In [12]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)

In [13]:
config_edit = config_util.get_configs_from_pipeline_file(PRETRAINED_MODEL_PATH+'/pipeline.config')

In [14]:
config_edit

{'model': ssd {
   num_classes: 4
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [15]:
# TRAINING

print('python models/research/object_detection/model_main_tf2.py --model_dir=workspace/models/model --pipeline_config_path=workspace/pre-trained-models/pipeline.config --num_train_steps=20000')

python models/research/object_detection/model_main_tf2.py --model_dir=workspace/models/model --pipeline_config_path=workspace/pre-trained-models/pipeline.config --num_train_steps=20000


In [5]:
%tensorboard --logdir ANNOTATION_PATH + '/anotasi/test.record'

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "D:\Additional Packages\miniconda3\envs\skripsi\lib\site-packages\google\auth\transport\requests.py", line 26, in <module>
    import requests
ModuleNotFoundError: No module named 'requests'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "D:\Additional Packages\miniconda3\envs\skripsi\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Additional Packages\miniconda3\envs\skripsi\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Additional Packages\miniconda3\envs\skripsi\Scripts\tensorboard.exe\__main__.py", line 4, in <module>
  File "C:\Users\anni\AppData\Roaming\Python\Python37\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "C:\Users\anni\AppData\Roaming\Python\Python37\site-packages\tensorboard